# Explicação 

Referência :  https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/

O processo de aprendizado por reforço envolve:

1. Observar o ambiente
2. Detectar como agir usando alguma estrategia
3. Agir de acordo com essa estrategia
4. Avaliar o resultado da ação e penalizar ou recompensar 
5. Aprender atraves das experiencias refinando nossa estrategia 
6. Iterar ate uma estrategia otima ser encontrada

O problema apresentado é uma simplicação de um táxi autônomo,
o objetivo do táxi é pegar um passageiro e o transportar para um determinado
local no menor tempo possivel, cuidando para não inflingir nenhuma regra de trafico. 

Dentro do contexto de RL (Reinforcement Learning) existe 3 conceitos principais:
    Ações: É o conjunto de ações que se pode realizar em determinada momento (estado).
    Para o caso do problema do taxi, existe 6 ações possiveis:
        1. ir para a dereita
        2. ir para a esquerda
        3. ir para tras
        4. ir para frente 
        5. Pagar o passageiro
        6. Deixar o passageiro

    Estado: No contexto de aprendizado por reforço o agente se encontra em um estado
    e deve realizar uma ação para ir para outro estado.

    Recompensas: É o valor dado ao agente pela sua ação em determinado estado, esse 
    valor é positivo caso a ação seja uma recompensa e negativo em caso de penalização.

O objetivo do aprendizado por reforço é encontrar uma sequencia de ações que maximize
a recompensa do agente.

No problema do táxi a seguir,onde existe 4 locais
e o objetivo do agente é pegar o passageiro em um local (representado pela cor azul) e transportar
para outro (representado pela cor rosa). O agente recebe +20 pontos caso ele consiga deixar o passageiro
no local exato e perde 1 ponto cada vez que ele realiza uma ação, há tambem uma penalidade de 10 pontos para o caso 
do agente pegar ou deixar o passageiro em um local errado.

# Código

In [1]:
# O quadrado (táxi) fica amarelo se o táxi esta sem passageiro e verde com passageiro.

import gym

env = gym.make("Taxi-v3").env
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [2]:
# O número de ações disponiveis para o táxi são 6, e o número de estados são as possiveis combinações da posição do carro
# posição do passageiro e posição de destino. O número de posições do carro são 25 já que existem 25 posições (grid 5x5)
# possiveis em que o caro pode ficar, já o número de posições possiveis para o passageiro são 4 (R, G, B, Y) mais um caso
# que é quando o carro pegou o passageiro, ou seja, temos 5 possibilidades para a possição do passageiro, já para a posição
# de destino temos 4 possibilidades (R, G, B, Y), o que nos leva a ter 25 * 5 * 4 = 500 possiveis estados para este ambiente.

env.reset() # reset environment to a new, random state
env.render()

print("Espaço de ações {}".format(env.action_space))
print("Espaço de estados {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Espaço de ações Discrete(6)
Espaço de estados Discrete(500)


In [3]:
# Criando um ambiente com a posição do táxi, passageiro e destino, atribuidos.
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("Estado:", state)

env.s = state
env.render()

Estado: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [4]:
# tabela de recompensa, atribuida por P, é uma lista que possui todos os possiveis estados e as suas respectivas ações.
# No problema do táxi cada ação é representada por um número de 0 a 5.
# {ação: [(probabilidade, proximo estado, recompensa, finalizado)]}.

env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

In [7]:
# Exemplo da resolução do problema de modo randômico. Onde a cada etapa uma ação random é tomada.
# Esse método dependendo da complexidade do problema pode ser inviável.

env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Número de etapas: {}".format(epochs))
print("Número de penalidades: {}".format(penalties))

Número de etapas: 1123
Número de penalidades: 349


In [6]:
# Mostra as ações do táxi de acordo com uma lista de frames.

from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.5)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)

Timestep: 132
State: 168
Action: 3
Reward: -1


KeyboardInterrupt: 

# Introdução ao Q-learning

O Q-learning é um algoritmo de apredizado por reforço para problemas não deterministicos que tenta se valer de exemplos deterministicos para a escolha da ação a ser tomada em determinado estado. Basicamente esse algoritmo busca criar uma tabela (chamada de Q-table) de consulta que mapeia a qualidade de uma ação em um determinada estado, ações que recebem uma recompensa são reforçadas, já ações que recebem punições são inibidas. O objetivo desse algoritmo é salvar o máximo de informações possíveis sobre o desempenho das ações em determinados estados, criando uma espécie de memoria auxiliar para o caso de quando essa ação for realizada em um estado parecido ou igual, existir a possibilidade de se consultar essa tabela. O algoritmo funciona da seguinte maneira:
    1. Inicializar a Q-table com zeros.
    2. Iniciar a exploração das ações para o estado atual, escolhendo uma ação dentre as possiveis para esse estado, essa
    escolha pode ser aleatória ou de acordo com o Q-value das ações para esse estado.
    3. Ir para o próximo estado de acordo com a ação tomada
    4. Para todas as ações possiveis nesse novo estado selecionar a que possui maior valor
    5. Atualizar a Q-table de acordo com a equação abaixo
    6. Selecionar esse novo estado como o atual.
    7. Até que o estado final seja alcançado repetir o passo 2.

Fórmula de atualização do valor de qualidade de uma determinada ação em um estado:

$Q(estado, ação) \gets (1-\alpha)Q(estado, ação) + \alpha(recompensa +  \gamma\max\limits_a Q(próximo\ estado, todas\ ações)$

Onde $\alpha$ é conhecido como learning rate (táxa de aprendizado) que diz com que velocidade o algoritmo vai convergir e para
o caso do aprendizado por reforço diz qual a importância da recompensa da ação mais recente sobre as próximas ações.

A recompensa é o valor de ganho recebido ao se realizar a ação.

O $\gamma$ é chamado fator de desconto e serve para determinar o quanto de importância o algoritmo deve dar a recompensas futuras. 

In [14]:
# iniciando a Q-tahle com todos os valores zeros.
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [8]:
# Treinando o algoritmo através de 100001 etapas onde cada etapa representa a conclusão de uma corrida do táxi.
# Primeiramente o algoritmo decide se vai usar uma ação randômica para o estado atual, ou se vai recorrer à Q-table.
# Após isso o par ação, estado, tem o seu peso atualizado de acordo com a formula de atualização mostrada acima.
# Esse processo continua até que o táxi tenha levado o seu cliente para o destino final.
# Esse algoritmo é rodado novamente para uma nova configuração.
# Observe que para esse algiritmo funcionar corretamente o mapeamento das ações e estados tem que ser realizado muitas vezes
# para capturar o máximo de diversidade possivel, isso torna esse algoritmo muitas vezes lento em ambientes complexos.

import random
import numpy as np
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

NameError: name 'q_table' is not defined

In [25]:
# Teste do táxi com a Q-table treinada anteriormente.
state = env.reset()
epochs, penalties, reward = 0, 0, 0
frames = []
done = False
while not done:
    action = np.argmax(q_table[state])
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 7
State: 410
Action: 5
Reward: 20
